In [2]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime

DATE_FORMAT = "%Y-%m-%d %H:%M:%S"

def send_json_data_from_url_to_kafka(topic, url, fields={}, addRequestDatetime=False):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    requestDatetime = datetime.now().strftime(DATE_FORMAT)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Iterate over each item in the list
        for item in data:
            # Extract the required fields
            message = {field: item.get(field) for field in fields}
            if addRequestDatetime:
                message["HeureRequete"] = requestDatetime
                
            
            # Send the message to Kafka
            producer.send(topic, message)
            print(f"Sent message: {message}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(data)} records.")
        producer.close()
    else:
        print(f"Failed to fetch data: {response.status_code}")


In [9]:
# https://open.tan.fr/doc/openapi#tag/Arrets/operation/getArretsHorairesTheoriques : 
arrets_fields = ["codeLieu", "libelle", "distance", "ligne"]

arrets_url = "https://open.tan.fr/ewp/arrets.json"

attente_fields = [
    'sens',
    'terminus',
    'infotrafic',
    'temps',
    'dernierDepart',
    'tempsReel',
    'ligne',
    'arret'
]

attente_url = "https://open.tan.fr/ewp/tempsattente.json/{}"

In [ ]:
send_json_data_from_url_to_kafka("arrets", arrets_url, arrets_fields)

In [ ]:
# Fetch the list of stops
response = requests.get(arrets_url)

# Iterate over each stop and send the waiting times to Kafka
for item in response.json():
    arret_code = item['codeLieu']
    send_json_data_from_url_to_kafka("attente", attente_url.format(arret_code), attente_fields, True)